In [ ]:
import torch
import torch.nn as tNN
import torch.linalg as tla
import numml.sparse as sp
import numml.nn as nNN
import numml.krylov as kry
import matplotlib.pyplot as plt
import sklearn.neighbors

In [ ]:
mu = torch.Tensor([
    [0.1, 0.1],
    [0.5, 0.9],
    [0.9, 0.2]
])
sigma = torch.Tensor([0.18, 0.18, 0.18])
N_c = 50

X = torch.cat([torch.randn(N_c, 2) * sigma[i] + mu[i] for i in range(len(mu))])
N = X.shape[0]

In [ ]:
plt.plot(X[:,0], X[:,1], 'o')

In [ ]:
G = sp.SparseCSRTensor(sklearn.neighbors.kneighbors_graph(X, 3))
plt.spy(G.to_scipy_csr(), markersize=2.)

In [ ]:
C = torch.abs(torch.randn((N, 3))) * 100
C.requires_grad = True

In [ ]:
def bin_argmax(C):
    Z = torch.zeros_like(C)
    v, i = torch.topk(C, k=1, dim=1)
    Z[torch.arange(C.shape[0]), torch.squeeze(i)] = torch.squeeze(v)
    return tNN.functional.softmax(Z, dim=1)

In [ ]:
optimizer = torch.optim.Adam([C], lr=0.1)

for i in range(100):
    optimizer.zero_grad()
    loss = (G@bin_argmax(C)).sum(dim=0).max()
    loss.backward()
    print(i, loss.item())